<a href="https://www.kaggle.com/code/abdelrahmanmfam/titanic-competition-pytorch-neural-network?scriptVersionId=175011170" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# **Load Dataset**

In [71]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/titanic/test.csv")

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# **Prepare Dataset**

Dropping the columnns: Name , Ticket , Cabin , Embarked

In [72]:
to_drop =['Name','Ticket','Cabin','Embarked','Age']
train_df = train_df.drop(columns = to_drop)
test_df = test_df.drop(columns = to_drop)

print(train_df.head())
print(test_df.head())

   PassengerId  Survived  Pclass     Sex  SibSp  Parch     Fare
0            1         0       3    male      1      0   7.2500
1            2         1       1  female      1      0  71.2833
2            3         1       3  female      0      0   7.9250
3            4         1       1  female      1      0  53.1000
4            5         0       3    male      0      0   8.0500
   PassengerId  Pclass     Sex  SibSp  Parch     Fare
0          892       3    male      0      0   7.8292
1          893       3  female      1      0   7.0000
2          894       2    male      0      0   9.6875
3          895       3    male      0      0   8.6625
4          896       3  female      1      1  12.2875


Turning the values of Sex to binary numerical values

In [73]:
train_df['Sex'] = train_df['Sex'].map({'male':0,'female':1})
test_df['Sex'] = test_df['Sex'].map({'male':0,'female':1})

train_df = train_df.astype(float)
test_df = test_df.astype(float)
print(train_df.head())
print(test_df.head())

   PassengerId  Survived  Pclass  Sex  SibSp  Parch     Fare
0          1.0       0.0     3.0  0.0    1.0    0.0   7.2500
1          2.0       1.0     1.0  1.0    1.0    0.0  71.2833
2          3.0       1.0     3.0  1.0    0.0    0.0   7.9250
3          4.0       1.0     1.0  1.0    1.0    0.0  53.1000
4          5.0       0.0     3.0  0.0    0.0    0.0   8.0500
   PassengerId  Pclass  Sex  SibSp  Parch     Fare
0        892.0     3.0  0.0    0.0    0.0   7.8292
1        893.0     3.0  1.0    1.0    0.0   7.0000
2        894.0     2.0  0.0    0.0    0.0   9.6875
3        895.0     3.0  0.0    0.0    0.0   8.6625
4        896.0     3.0  1.0    1.0    1.0  12.2875


Splitting the Data into training and validation sets

# Normalize the Data


In [74]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

col_to_normalize = ['Fare']
train_df[col_to_normalize] = scaler.fit_transform(train_df[col_to_normalize])
test_df[col_to_normalize] = scaler.fit_transform(test_df[col_to_normalize])
test_df.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Fare
0,892.0,3.0,0.0,0.0,0.0,0.015282
1,893.0,3.0,1.0,1.0,0.0,0.013663
2,894.0,2.0,0.0,0.0,0.0,0.018909
3,895.0,3.0,0.0,0.0,0.0,0.016908
4,896.0,3.0,1.0,1.0,1.0,0.023984


# Splitting the data

In [75]:
import torch
from sklearn.model_selection import train_test_split

# Select features and target variable
X = train_df[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']].values  # Features
y = train_df['Survived'].values  # Target variable
X_test = test_df[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare']].values 

# Split data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1) 

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
print(X_train_tensor.size())
print(X_test)

torch.Size([712, 5])
[[3.         0.         0.         0.         0.01528158]
 [3.         1.         1.         0.         0.01366309]
 [2.         0.         0.         0.         0.01890874]
 ...
 [3.         0.         0.         0.         0.01415106]
 [3.         0.         0.         0.         0.01571255]
 [3.         0.         1.         1.         0.0436405 ]]


# Creating a TensorDataset


In [76]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(X_train_tensor,y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor)
print(X_train_tensor.size())
print(X_val_tensor.size())
print(X_test_tensor.size())

torch.Size([712, 5])
torch.Size([179, 5])
torch.Size([418, 5])


# **Train Model**


## Creating the Model

In [77]:
import torch.nn as nn
import torch.nn.functional as F



class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 1)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sig(x)
        
        return x

        
input_size = X_train.shape[1]

model = Net()
print(model)

Net(
  (fc1): Linear(in_features=5, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
  (relu): ReLU()
  (sig): Sigmoid()
)


## Define the loss and optimizer

In [78]:
learning_rate = 0.0001
criterion  = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training Loop

In [79]:
#Creating DataLoaders
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset,batch_size = 16, shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = 16, shuffle = True)
test_loader = DataLoader(test_dataset,batch_size = 10000, shuffle = True)

In [80]:
num_epoch = 200
for epoch in range(num_epoch):
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
#         print(outputs,labels)
        loss = criterion(outputs, labels)
#         print(loss)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Print average loss for the epoch
    if(epoch%5 == 0):
        print(f"Epoch [{epoch + 1}/{num_epoch}], Loss: {running_loss/ len(train_loader)}")

print("Training finished!")

Epoch [1/200], Loss: 0.7208981010648939
Epoch [6/200], Loss: 0.6788648724555969
Epoch [11/200], Loss: 0.6380788577927483
Epoch [16/200], Loss: 0.6044656780030992
Epoch [21/200], Loss: 0.5686879919634925
Epoch [26/200], Loss: 0.5311755332681868
Epoch [31/200], Loss: 0.4984360370371077
Epoch [36/200], Loss: 0.4798160228464339
Epoch [41/200], Loss: 0.4679371221197976
Epoch [46/200], Loss: 0.4641225987010532
Epoch [51/200], Loss: 0.456319436762068
Epoch [56/200], Loss: 0.45262212687068515
Epoch [61/200], Loss: 0.451876864168379
Epoch [66/200], Loss: 0.45338559746742246
Epoch [71/200], Loss: 0.4535328182909224
Epoch [76/200], Loss: 0.4531958374712202
Epoch [81/200], Loss: 0.4458098673158222
Epoch [86/200], Loss: 0.44915464520454407
Epoch [91/200], Loss: 0.4454838424921036
Epoch [96/200], Loss: 0.44829164809650845
Epoch [101/200], Loss: 0.4446454892555873
Epoch [106/200], Loss: 0.4411719083786011
Epoch [111/200], Loss: 0.4450006494919459
Epoch [116/200], Loss: 0.43998887240886686
Epoch [121/

In [81]:
# Set model to evaluation mode
model.eval()

# Iterate over evaluation dataset
total_samples = 0
correct_predictions = 0
total_loss = 0.0
with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in val_loader:  # Iterate over batches
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        print(type(inputs))
        # Compute evaluation metrics
        _, predicted = torch.max(outputs, 1)  # Get class predictions
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).size(0)
        
        total_loss += loss.item() * labels.size(0)

# Compute accuracy and average loss
print(correct_predictions)
print(total_samples)
accuracy = correct_predictions / total_samples
average_loss = total_loss / total_samples

print(f"Accuracy: {accuracy:.4f}, Average Loss: {average_loss:.4f}")

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
179
179
Accuracy: 1.0000, Average Loss: 0.4389


In [82]:
with torch.no_grad():
    for inputs in test_loader:
        print(inputs[0].size())
        output = model(inputs[0])
output = (output < 0.5)
print(output)

torch.Size([418, 5])
tensor([[ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [F

In [92]:
import csv

submission = [['PassengerId', 'Survived']]
for i in range(len(output)):
    result = 0
    if output[i]:
        result = 1
    submission.append([int(test_df['PassengerId'][i]), result])

submission

[['PassengerId', 'Survived'],
 [892, 1],
 [893, 0],
 [894, 1],
 [895, 0],
 [896, 0],
 [897, 1],
 [898, 0],
 [899, 0],
 [900, 0],
 [901, 1],
 [902, 0],
 [903, 0],
 [904, 1],
 [905, 1],
 [906, 0],
 [907, 1],
 [908, 0],
 [909, 1],
 [910, 1],
 [911, 0],
 [912, 0],
 [913, 0],
 [914, 1],
 [915, 1],
 [916, 1],
 [917, 0],
 [918, 1],
 [919, 1],
 [920, 1],
 [921, 1],
 [922, 0],
 [923, 1],
 [924, 0],
 [925, 0],
 [926, 1],
 [927, 1],
 [928, 1],
 [929, 1],
 [930, 1],
 [931, 0],
 [932, 0],
 [933, 0],
 [934, 1],
 [935, 0],
 [936, 1],
 [937, 0],
 [938, 0],
 [939, 1],
 [940, 1],
 [941, 0],
 [942, 1],
 [943, 1],
 [944, 0],
 [945, 1],
 [946, 0],
 [947, 0],
 [948, 0],
 [949, 0],
 [950, 1],
 [951, 1],
 [952, 1],
 [953, 1],
 [954, 1],
 [955, 1],
 [956, 1],
 [957, 1],
 [958, 0],
 [959, 0],
 [960, 1],
 [961, 1],
 [962, 1],
 [963, 0],
 [964, 0],
 [965, 1],
 [966, 0],
 [967, 1],
 [968, 1],
 [969, 1],
 [970, 1],
 [971, 1],
 [972, 0],
 [973, 1],
 [974, 0],
 [975, 0],
 [976, 0],
 [977, 1],
 [978, 1],
 [979, 0],
 [

In [93]:
with open('submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(submission)
    
print('Writing Complete!')

Writing Complete!
